In [ ]:
# ================== SCRIPT 5 — Seurat + SingleR + GSEA ==================
#  Input:  /home/jovyan/work/Data/{matrix.mtx.gz, features.tsv.gz, barcodes.tsv.gz}
#  Output: /home/jovyan/work/Results/script5/*

# ================== CONFIG ==================
base_dir    <- "/home/jovyan/work"
data_dir    <- file.path(base_dir, "Data")
results_dir <- file.path(base_dir, "Results", "script5")
dir.create(results_dir, showWarnings = FALSE, recursive = TRUE)

matrix_file   <- file.path(data_dir, "matrix.mtx.gz")
features_file <- file.path(data_dir, "features.tsv.gz")
barcodes_file <- file.path(data_dir, "barcodes.tsv.gz")

seed <- 1337
set.seed(seed)

# ================== LIBRARIES (minime) ==================
suppressPackageStartupMessages({
  library(Seurat)
  library(SingleR)
  library(celldex)
  library(SingleCellExperiment)
  library(clusterProfiler)
  library(msigdbr)
  library(org.Hs.eg.db)
  library(Matrix)
  library(ggplot2)
})

# ================== FUNZIONI UTILI ==================
save_plot_safe <- function(p, file, width=8, height=6, dpi=300){
  ok <- tryCatch({
    ggplot2::ggsave(filename = file, plot = p, width = width, height = height, dpi = dpi, device = "png")
    TRUE
  }, error = function(e) FALSE)
  if(!ok){
    png(file, width = width*300, height = height*300, res = dpi)
    print(p)
    dev.off()
  }
}

# ================== LOAD 10X (features = ENSEMBL) ==================
mat <- ReadMtx(
  mtx      = matrix_file,
  features = features_file,
  cells    = barcodes_file,
  feature.column = 1 # ENSEMBL ID (anche con .versione)
)

# ---- ENSEMBL -> SYMBOL con aggregazione sparsa ----
ens_raw   <- rownames(mat)
ens_clean <- sub("\\..*$", "", ens_raw)     # rimuove suffisso .v
rownames(mat) <- ens_clean

map <- suppressWarnings(
  bitr(unique(ens_clean), fromType = "ENSEMBL", toType = "SYMBOL", OrgDb = org.Hs.eg.db)
)
sym_vec <- map$SYMBOL[match(ens_clean, map$ENSEMBL)]

keep     <- !is.na(sym_vec) & sym_vec != ""
mat_keep <- mat[keep, , drop = FALSE]
sym_keep <- sym_vec[keep]

grp <- droplevels(factor(sym_keep))
MM  <- Matrix::sparse.model.matrix(~ 0 + grp)     # n_geni_keep x n_symbol_unici
mat_symbol <- Matrix::t(MM) %*% mat_keep          # righe = SYMBOL unici
rownames(mat_symbol) <- levels(grp)

message(sprintf("Mapped to SYMBOL: %d / %d (%.1f%%)",
                nrow(mat_symbol), length(ens_clean),
                100 * nrow(mat_symbol)/length(ens_clean)))

# ================== SEURAT: PREPROCESS ==================
obj <- CreateSeuratObject(counts = mat_symbol, project = "sample")
obj <- NormalizeData(obj, verbose = FALSE)
obj <- FindVariableFeatures(obj, verbose = FALSE)
obj <- ScaleData(obj, verbose = FALSE)
obj <- RunPCA(obj, verbose = FALSE)
obj <- FindNeighbors(obj, dims = 1:20, verbose = FALSE)
obj <- FindClusters(obj, resolution = 0.5, verbose = FALSE)
obj <- RunUMAP(obj, dims = 1:20, verbose = FALSE, seed.use = seed)

# ================== SingleR: reference umana migliore ==================
options(ExperimentHub.localHub = TRUE)

sce <- as.SingleCellExperiment(obj)
if (!"logcounts" %in% assayNames(sce)) {
  assay(sce, "logcounts") <- GetAssayData(obj, slot = "data")
}

ref_fns <- list(
  function() celldex::HumanPrimaryCellAtlasData(),
  function() celldex::BlueprintEncodeData(),
  function() celldex::MonacoImmuneData(),
  function() celldex::NovershternHematopoieticData(),
  function() celldex::DatabaseImmuneCellExpressionData()
)

best_ref <- NULL
best_overlap <- -1L
for (f in ref_fns) {
  ref_try <- tryCatch(f(), error = function(e) NULL)
  if (is.null(ref_try)) next
  ov <- length(intersect(rownames(sce), rownames(ref_try)))  # overlap in SYMBOL
  if (ov > best_overlap) {
    best_overlap <- ov
    best_ref <- ref_try
  }
}

have_singleR <- FALSE
if (!is.null(best_ref) && best_overlap >= 50) {
  message(sprintf("SingleR: uso reference con overlap = %d geni.", best_overlap))
  lab_col <- if ("label.main" %in% colnames(SummarizedExperiment::colData(best_ref))) {
    "label.main"
  } else if ("label.fine" %in% colnames(SummarizedExperiment::colData(best_ref))) {
    "label.fine"
  } else {
    NULL
  }
  if (!is.null(lab_col)) {
    singler <- SingleR(
      test = sce,
      ref = best_ref,
      labels = SummarizedExperiment::colData(best_ref)[[lab_col]]
    )
    obj$SingleR <- singler$labels
    have_singleR <- TRUE
  } else {
    message("Reference scelta senza 'label.main'/'label.fine' → SingleR SKIPPED")
  }
} else {
  message("SingleR SKIPPED: nessuna reference con overlap sufficiente.")
}

# ================== UMAP PLOTS (no patchwork break) ==================
# Usa combine=FALSE per ottenere ggplot puro e salvare senza errori
p_clusters_list <- DimPlot(
  obj, group.by = "seurat_clusters",
  label = TRUE, repel = TRUE,
  combine = FALSE
)
p_clusters <- p_clusters_list[[1]] + ggtitle("UMAP — Seurat clusters") +
  theme(legend.position = "right")
save_plot_safe(p_clusters, file.path(results_dir, "UMAP_clusters.png"))

if (have_singleR) {
  p_annot_list <- DimPlot(
    obj, group.by = "SingleR",
    label = TRUE, repel = TRUE,
    combine = FALSE
  )
  p_annot <- p_annot_list[[1]] + ggtitle("UMAP — SingleR annotation") +
    theme(legend.position = "right")
  save_plot_safe(p_annot, file.path(results_dir, "UMAP_SingleR.png"))
}

# ================== GSEA (Hallmark, SYMBOL) per cluster ==================
msig <- msigdbr(species = "Homo sapiens", category = "H")
TERM2GENE <- msig[, c("gs_name", "gene_symbol")]
colnames(TERM2GENE) <- c("term", "gene")

gsea_dir <- file.path(results_dir, "GSEA_Hallmark_byCluster_SYMBOL")
dir.create(gsea_dir, showWarnings = FALSE, recursive = TRUE)

clusters <- levels(Idents(obj))
for (cl in clusters) {
  de <- tryCatch(
    FindMarkers(obj, ident.1 = cl, only.pos = FALSE, logfc.threshold = 0,
                min.pct = 0, verbose = FALSE),
    error = function(e) NULL
  )
  if (is.null(de) || nrow(de) == 0) next

  de$SYMBOL <- rownames(de)
  de <- de[de$SYMBOL %in% unique(TERM2GENE$gene) & !is.na(de$avg_log2FC), , drop = FALSE]
  if (nrow(de) < 10) next

  gene_list <- de$avg_log2FC
  names(gene_list) <- de$SYMBOL
  gene_list <- sort(gene_list, decreasing = TRUE)

  gsea_res <- tryCatch(
    GSEA(geneList = gene_list, TERM2GENE = TERM2GENE, verbose = FALSE),
    error = function(e) NULL
  )
  if (!is.null(gsea_res)) {
    out_csv <- file.path(gsea_dir, sprintf("GSEA_cluster%s_vs_all.csv", cl))
    write.csv(as.data.frame(gsea_res), out_csv, row.names = FALSE)
  }
}

# ================== SAVE OBJECTS + LOG ==================
saveRDS(obj, file.path(results_dir, "seurat_obj_script5.rds"))

msg <- c(
  paste0("UMAP clusters: ", file.path(results_dir, "UMAP_clusters.png")),
  if (have_singleR) paste0("UMAP SingleR: ", file.path(results_dir, "UMAP_SingleR.png")) else "UMAP SingleR: SKIPPED",
  paste0("GSEA (Hallmark, SYMBOL) per cluster in: ", gsea_dir),
  paste0("Seurat object: ", file.path(results_dir, "seurat_obj_script5.rds"))
)
message(paste(msg, collapse = "\n"))


'select()' returned 1:many mapping between keys and columns

Mapped to SYMBOL: 24229 / 36753 (65.9%)

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
SingleR SKIPPED: nessuna reference con overlap sufficiente.

Warning message:
“The `category` argument of `msigdbr()` is deprecated as of msigdbr 10.0.0.
ℹ Please use the `collection` argument instead.”
